קודם נריץ את התא הזה:

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
driver.get("https://www.yad2.co.il/vehicles/private-cars")
df = pd.DataFrame()
df

אחר כך נכנס לדף מסוים 
ונריץ את הקוד הזה:

In [ ]:
pageList = (driver.find_elements_by_class_name("page-num"))
print ("Number of total pages is: ")
print(pageList[-1].text)
maxpageNum = int(pageList[-1].text)
mainpage = driver.find_element_by_tag_name("body")

for page in range (1, maxpageNum - 2):
    try:
        time.sleep(10)
        # for every page do the following:
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "inner_content"))) #wait till the table is showing
        print ("number of page showing:", end =" ")
        print(page+1)

        # find all ads by xpath
        ads = driver.find_elements_by_xpath("//div[@class='feeditem table']")
        print ("number of ads found:", end =" ")
        print(len(ads))

        # did u find any ads?
        if len(ads)==0:
            print("didn't find any ads...")
            element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, "הבא")))
            pageclick = driver.find_element_by_link_text("הבא")
            element.click()
            continue

        #if u did then do this:
        else:
            ###click on all ads
            for ad in ads:
                ad.click()
                time.sleep(0.1)
                mainpage.send_keys(Keys.PAGE_DOWN)
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "dt")))
                time.sleep(0.1)

            try:
                for ad in ads:
                ### get all the fundamental vals into the singular car dictionary
                    try:
                    #get type
                        car_type = ad.find_element_by_class_name("title") 
                        car_dict = {}
                        car_dict['סוג'] = car_type.text

                    ##get the year, hand and volume

                        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "key")))
                        keys = ad.find_elements_by_class_name("key")
                        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "val")))
                        vals = ad.find_elements_by_class_name("val")

                        for key, val in zip(keys, vals):
                            car_dict[key.text] = val.text
                            
                    #get price
                        price = ad.find_element_by_class_name("price")
                        car_dict["מחיר"] = price.text
                        

                    #get all the details:

                        element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "details_wrapper")))
                        details = ad.find_element_by_class_name("details_wrapper")
                        keys = details.find_elements_by_tag_name("dt")
                        vals = details.find_elements_by_tag_name("dd")

                        for key, val in zip(keys, vals):
                            car_dict[key.text] = val.text

                    #get air pollution rating
                        try:
                            pollution = ad.find_element_by_class_name("ad_pollution")
                            pollution_rating = pollution.find_element_by_class_name("selected")
                            car_dict["זיהום מירבי"] = pollution_rating.text
                        except:
                            car_dict["זיהום מירבי"] = "NaN"

                    #get area
                        upper_info = ad.find_element_by_class_name("upper_info")
                        contents = upper_info.find_elements_by_tag_name("div")
                        for content in contents:
                            key = content.find_element_by_tag_name("span")
                            val = content.find_element_by_class_name("upper_info_text")
                            car_dict[key.text] = val.text


                        #append the car to the dataframe
                        df = df.append(car_dict, ignore_index=True)
                    except:
                        
                        print("internal ad exception; skipping...")
                        continue
            except:
                print("external ad exception; skipping...")
                continue
            print("DONE Page no.", end = " ")
            print(page+1)
            print("Number of ads scraped (approx.):", end = " ")
            print(df.shape[0])

            


            element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, "הבא")))
            pageclick = driver.find_element_by_link_text("הבא")
            element.click()
    except:
        print("page exception/CAPTCHA; sleeping and then skipping...", end = " ")
        time.sleep(30)
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, "הבא")))
        pageclick = driver.find_element_by_link_text("הבא")
        element.click()
        continue
    
            

נריץ כדי לראות את הדאטה פריים:

In [1]:
df

NameError: name 'df' is not defined

בסיום כל ריצה תשמור את הדאטה פריים לקובץ אקסל:

In [ ]:
df.to_csv("Yad2Crawler99.csv")